<a href="https://colab.research.google.com/github/dzastin96/product-category-classifier/blob/main/notebooks/product_category_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>